In [1]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
import pandas as pd

In [3]:
!head usgs_fault -n 12

"head" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [4]:
df = pd.read_csv('usgs.csv', comment='#', skiprows=9, names='Lat. Lon. depth slip rake strike dip t_rup t_ris t_fal mo'.split(' '))
df.head()

,Lat.,Lon.,depth,slip,rake,strike,dip,t_rup,t_ris,t_fal,mo
0,-31.5677,-72.5003,2.069,348.92090,101.64070,6,19,48.8,4.8,6.4,3.500000e+25
1,-31.4603,-72.4871,2.069,292.78680,89.71340,6,19,31.2,1.6,1.6,2.940000e+25
2,-31.3529,-72.4738,2.069,266.65881,99.74409,6,19,38.4,1.6,1.6,2.680000e+25
3,-31.2455,-72.4605,2.069,279.72589,89.97067,6,19,38.4,1.6,1.6,2.810000e+25
4,-31.1381,-72.4473,2.069,313.14539,86.60627,6,19,47.2,4.8,1.6,3.140000e+25


In [5]:
df = df.rename(columns={
    'Lat.': 'lat',
    'Lon.': 'lon'
})
df['L'] = 12000
df['W'] = 8800
df['depth']*=1000
df['reference'] = 'center'
df['slip'] *= 0.01
# df['lon'] +=360
df.head()

,lat,lon,depth,slip,rake,strike,dip,t_rup,t_ris,t_fal,mo,L,W,reference
0,-31.5677,-72.5003,2069.0,3.489209,101.64070,6,19,48.8,4.8,6.4,3.500000e+25,12000,8800,center
1,-31.4603,-72.4871,2069.0,2.927868,89.71340,6,19,31.2,1.6,1.6,2.940000e+25,12000,8800,center
2,-31.3529,-72.4738,2069.0,2.666588,99.74409,6,19,38.4,1.6,1.6,2.680000e+25,12000,8800,center
3,-31.2455,-72.4605,2069.0,2.797259,89.97067,6,19,38.4,1.6,1.6,2.810000e+25,12000,8800,center
4,-31.1381,-72.4473,2069.0,3.131454,86.60627,6,19,47.2,4.8,1.6,3.140000e+25,12000,8800,center


In [6]:
df.to_csv('earthquake.csv',index=False)